In [100]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np


In [101]:
# Step 1: Load and preprocess the data
# Load the CSV files
symptom_severity_df = pd.read_csv('symptom-severity.csv')
disease_df = pd.read_csv('dataset.csv')
disease_description_df = pd.read_csv('disease_Description.csv')
disease_precaution_df = pd.read_csv('disease_precaution.csv')


In [102]:
disease_symptom_map = {}
for index, row in disease_df.iterrows():
    disease = row['Disease']
    # Check for NaN values and filter out non-null symptoms
    symptoms = [s.strip() for s in row if pd.notna(s) and s.strip()]
    disease_symptom_map[disease] = symptoms

In [103]:
merged_data = disease_df.merge(disease_description_df, on='Disease').merge(disease_precaution_df, on='Disease')

# Create label encoders for Disease and Symptom
disease_label_encoder = LabelEncoder()
symptom_label_encoder = LabelEncoder()

# Fit the symptom_label_encoder with unique symptoms
unique_symptoms = symptom_severity_df['Symptom'].unique()
symptom_label_encoder.fit(unique_symptoms)


LabelEncoder()

In [104]:

merged_data['Disease_Code'] = disease_label_encoder.fit_transform(merged_data['Disease'])


In [105]:
symptom_weight_map = dict(zip(symptom_severity_df['Symptom'], symptom_severity_df['weight']))

In [106]:
unique_disease_labels = merged_data['Disease_Code'].unique()
print(unique_disease_labels,merged_data['Disease_Code'])

[13  4 14  9 12 30  1 15  6 27  7 29 25 26  8 11 34 37 17 18 19 20  3 33
 10 31 16 36 23 21 22 28  5  0  2 35 32 24] 0       13
1       13
2       13
3       13
4       13
        ..
4555    24
4556    24
4557    24
4558    24
4559    24
Name: Disease_Code, Length: 4560, dtype: int32


In [107]:
default_weight = 1 
X = [sum([symptom_weight_map.get(symptom, default_weight) for symptom in symptoms]) for symptoms in disease_symptom_map.values()]

In [111]:
y = merged_data['Disease_Code']
print(len(y),len(X),X)


4560 41 [[10]
 [17]
 [29]
 [27]
 [17]
 [28]
 [22]
 [42]
 [19]
 [29]
 [22]
 [33]
 [24]
 [15]
 [32]
 [35]
 [45]
 [56]
 [51]
 [46]
 [44]
 [26]
 [37]
 [63]
 [33]
 [76]
 [78]
 [53]
 [27]
 [20]
 [33]
 [54]
 [44]
 [48]
 [21]
 [19]
 [28]
 [10]
 [18]
 [16]
 [20]]


In [109]:
X = np.array(X).reshape(-1, 1)
# Create a random forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

ValueError: Found input variables with inconsistent numbers of samples: [41, 4560]

In [ ]:
# Step 4: Create a function to predict the disease
def predict_disease(symptoms):
    encoded_symptoms = [symptom_label_encoder.transform([symptom])[0] for symptom in symptoms]
    severity = sum([symptom_severity_df['weight'][i] for i in encoded_symptoms])
    disease_code = model.predict([[severity]])[0]
    predicted_disease = disease_label_encoder.inverse_transform([disease_code])[0]

    disease_info = merged_data[merged_data['Disease'] == predicted_disease]
    disease_description = disease_info['Description'].values[0]
    precautions = disease_info[['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].values[0]

    return predicted_disease, disease_description, precautions


In [ ]:
input_symptoms = ['itching', 'skin_rash', 'nodal_skin_eruptions']
predicted_disease, disease_description, precautions = predict_disease(input_symptoms)
print(f'Predicted Disease: {predicted_disease}')
print(f'Disease Description: {disease_description}')
print(f'Precautions: {precautions}')

NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.